In [45]:
import numpy as np
import matplotlib.pyplot as plt

m=50000

In [46]:
def load_data(filename):
    
    import pickle

    with open(filename, 'rb') as f:
        datadict = pickle.load(f, encoding='bytes')
        x=datadict[b'data']
        x=x.reshape(10000,3072)
        y=datadict[b'labels']
        y=np.array(y)
        return x,y

def get_data(arr):
    xdata=[]
    ydata=[]
    for i in arr:
        a,b=load_data(i)
        xdata.append(a)
        ydata.append(b)
        del a,b
    xdata=np.concatenate(xdata)
    ydata=np.concatenate(ydata)

    xdata=xdata/255
    return xdata,ydata

def load_data_test(filename):
    
    import pickle

    with open(filename, 'rb') as f:
        datadict = pickle.load(f, encoding='bytes')
        xtest=np.asarray(datadict[b'data'])
        xtest=xtest.reshape(10000,3072)
        ytest=np.asarray(datadict[b'labels'])
        return xtest/255, ytest




paths=["../input/cifar10-python/cifar-10-batches-py/data_batch_1","../input/cifar10-python/cifar-10-batches-py/data_batch_2","../input/cifar10-python/cifar-10-batches-py/data_batch_3","../input/cifar10-python/cifar-10-batches-py/data_batch_4","../input/cifar10-python/cifar-10-batches-py/data_batch_5"]
xdata, ydata=get_data(paths)

xtest, ytest = load_data_test("../input/cifar10-python/cifar-10-batches-py/test_batch")

xdata = xdata.T

xtest = xtest.T


In [47]:
def relu(z):
    return np.maximum(z, 0)

def softmax(z):
    a = np.exp(z) / sum(np.exp(z))
    return a

def drelu(z):
    return z > 0

def one_hot(y):
    y1 = np.zeros((y.size, y.max() + 1))
    y1[np.arange(y.size), y] = 1
    y1 = y1.T
    return y1

In [48]:
def parameters():
    w1 = np.random.rand(100, 3072)-0.5
    b1 = np.random.rand(100, 1)-0.5
    w2 = np.random.rand(10, 100)-0.5
    b2 = np.random.rand(10, 1)-0.5
    return w1, b1, w2, b2

def forward_propogation(w1, b1, w2, b2, x):
    z1 = w1.dot(x) + b1
    a1 = relu(z1)
    z2 = w2.dot(a1) + b2
    a2 = softmax(z2)
    return z1, a1, z2, a2
def backward_propogation(z1, a1, z2, a2, w1, w2, x, y):
    y1 = one_hot(y)
    dz2 = a2 - y1
    dw2 = 1 / m * dz2.dot(a1.T)
    db2 = 1 / m * np.sum(dz2)
    dz1 = w2.T.dot(dz2) * drelu(z1)
    dw1 = 1 / m * dz1.dot(x.T)
    db1 = 1 / m * np.sum(dz1)
    return dw1, db1, dw2, db2

def update_parameters(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha):
    w1 = w1 - alpha * dw1
    b1 = b1 - alpha * db1    
    w2 = w2 - alpha * dw2  
    b2 = b2 - alpha * db2    
    return w1, b1, w2, b2

In [49]:
def predictions(a2):
    return np.argmax(a2, 0)

def accuracy(predictions, y):
    print(predictions, y)
    return np.sum(predictions == y) / y.size

In [50]:
def gradient_descent(x, y, alpha, iterations):
    w1, b1, w2, b2 = parameters()
    for i in range(iterations):
        z1, a1, z2, a2 = forward_propogation(w1, b1, w2, b2, x)
        dw1, db1, dw2, db2 = backward_propogation(z1, a1, z2, a2, w1, w2, x, y)
        w1, b1, w2, b2 = update_parameters(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            preds = predictions(a2)
            print(accuracy(preds, y))
    return w1, b1, w2, b2

In [53]:
w1, b1, w2, b2 = gradient_descent(xtest, ytest, 0.1, 100)

In [ ]:
def make_predictions(x, w1, b1, w2, b2):
    _, _, _, a2 = forward_propogation(w1, b1, w2, b2, x)
    preds = predictions(a2)
    return preds

make_predictions = make_predictions(xtest, w1, b1, w2, b2)
accuracy(make_predictions, ytest)